In [1]:
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import pandas as pd


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
pytrends = TrendReq(hl='en-US', tz=360)


In [3]:
num_keywords = min(25, int(input("Enter the number of keywords (up to 25): ")))
keywords = [input(f"Enter keyword {i+1}: ") for i in range(num_keywords)]


Enter the number of keywords (up to 25): 5
Enter keyword 1: ferrari
Enter keyword 2: nissan
Enter keyword 3: bmw
Enter keyword 4: hilary
Enter keyword 5: trump


In [4]:
pytrends.build_payload(keywords, timeframe='today 5-y', geo='', gprop='')
interest_over_time = pytrends.interest_over_time()


In [ ]:
import plotly.graph_objects as go

def plot_interest_over_time(data, keys):
    fig = go.Figure()
    
    for key in keys:
        if key in data.columns:
            fig.add_trace(go.Scatter(x=data.index, y=data[key], mode='lines', name=key, hoverinfo='x+y+name'))
    
    fig.update_layout(
        title='Google Trends Over Time',
        xaxis_title='Date',
        yaxis_title='Interest',
        xaxis=dict(showline=True, showgrid=True, showticklabels=True, linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside', tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)')),
        yaxis=dict(showgrid=True, zeroline=False, showline=False, showticklabels=True),
        autosize=True,
        margin=dict(autoexpand=True, l=100, r=20, t=110),
        showlegend=True
    )

    fig.show()

plot_interest_over_time(interest_over_time, keywords)


In [6]:
# Ensure only numeric columns are considered
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')  # Dropping the 'isPartial' column if present
highest_interest = numeric_data.max()
highest_interest_keywords = highest_interest.idxmax() if not highest_interest.empty else "No data available"

time_intervals = ['1w', '1m', '3m', '6m', '12m']
percent_changes = {}

for interval in time_intervals:
    interval_data = interest_over_time.last(interval).iloc[0]
    # Calculate percentage change for each keyword
    percent_change = (interval_data - numeric_data.iloc[0]) / numeric_data.iloc[0] * 100
    percent_changes[interval] = percent_change

# Convert the dictionary to a DataFrame
percent_change_df = pd.DataFrame(percent_changes)
percent_change_df.head()


,1w,1m,3m,6m,12m
bmw,28.571429,14.285714,14.285714,0.0,0.0
ferrari,0.0,0.0,0.0,0.0,0.0
hilary,inf,NaN,NaN,NaN,NaN
isPartial,NaN,NaN,NaN,NaN,NaN
nissan,-20.0,-20.0,-20.0,-20.0,-20.0


In [7]:
import plotly.express as px

# Drop the 'isPartial' column from numeric_data
numeric_data = interest_over_time.drop(columns=['isPartial'], errors='ignore')

# Drop the 'isPartial' column from percent_change_df
percent_change_df = percent_change_df.dropna()
# Create a heatmap using Plotly Express
fig = px.imshow(percent_change_df,
                x=percent_change_df.columns,
                y=percent_change_df.index,
                color_continuous_scale='reds',
                title='Percentage Increase in Interest Over Time Intervals - Heatmap')

# Update the layout
fig.update_layout(
    xaxis=dict(side='top'),
    xaxis_title='Keywords',
    yaxis_title='Time Intervals',
    autosize=True,
    margin=dict(autoexpand=True, l=100, r=20, t=110),
    showlegend=True
)

# Show the plot
fig.show()


In [11]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Function to scrape web content and analyze related topics
def find_related_topics(keywords):
    related_topics = Counter()
    
    for keyword in keywords:
        search_url = f"https://www.google.com/search?q={keyword.replace(' ', '+')}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract text content from the search results page
        text_content = " ".join([p.get_text() for p in soup.find_all('p')])
        
        # Tokenize and remove stopwords
        tokens = word_tokenize(text_content)
        filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
        
        # Update related topics counter
        related_topics.update(filtered_tokens)
    
    return related_topics

# Find and display related topics
related_topics = find_related_topics(keywords)
print("\nRelated Topics:")
for topic, count in related_topics.most_common(10):
    print(f"{topic}: {count}")



Related Topics:


In [12]:
import requests
from bs4 import BeautifulSoup
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# User input for keywords
num_keywords = min(25, int(input("Enter the number of keywords (up to 25): ")))
keywords = [input(f"Enter keyword {i+1}: ") for i in range(num_keywords)]

# Function to scrape web content
def scrape_web_content(keyword):
    search_url = f"https://www.google.com/search?q={keyword.replace(' ', '+')}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text_content = " ".join([p.get_text() for p in soup.find_all('p')])
    return text_content

# Preprocess and analyze web content
documents = [scrape_web_content(keyword) for keyword in keywords]
texts = [[word for word in word_tokenize(doc.lower()) if word.isalnum() and word not in stopwords.words('english')] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

# Print identified topics
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}: {topic}")


ModuleNotFoundError: No module named 'gensim'

In [14]:
pip install gensim

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized

Note: you may need to restart the kernel to use updated packages.
